In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(parent_dir)
sys.path.append(parent_dir)

c:\Users\cyon1\Desktop\LLM_cls_sum


In [2]:
from pprint import pprint

In [4]:
# llama_api.py
import os, sys
## 상위 경로 추가
from config.prompt.generate_prompt import generate_prompt
from source.utils import check_gpu_allocation

from dotenv import load_dotenv
import torch
from transformers import pipeline

# Load environment variables
load_dotenv()
api_token = os.getenv("HUGGINGFACE_API_TOKEN")

## 모델 parameter setting 및 모델 로드
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    do_sample=False,
    top_p = None,
    temperature = None
    # trust_remote_code = True
)

## 현재 GPU 스펙 및 allocation 현황 조회
check_gpu_allocation()


c:\Users\cyon1\Desktop\project\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.32s/it]
c:\Users\cyon1\Desktop\project\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Device 0: NVIDIA GeForce RTX 4090 Laptop GPU - Allocated: 6128.35 MB


In [5]:
def summarize_document(document: list):

    # print(f'for debug: {document}')
    messages = []
    for text in document:
        generated_prompt = generate_prompt(text, "summarize")
        message = [{"role": "system", "content": generated_prompt["role"]}, 
                    {"role": "user", "content": generated_prompt["message"]}]
        # print(f"debug: {message}")
        messages.append(message)
    # print(f"debug: {len(messages)}")
    try:
        outputs = pipe(
            messages,
            max_new_tokens=12800,
        )
        response_message = [output[0]["generated_text"][-1]["content"] for output in outputs] 
        return {"response": response_message}
    except Exception as e:
        raise Exception(str(e))
    


def classify_topic(document: list):
    messages = []
    for text in document:
        generated_prompt = generate_prompt(text, "classify")
        # print(generated_prompt["message"])
        message = [{"role": "system", "content": generated_prompt["role"]}, 
                    {"role": "user", "content": generated_prompt["message"]}]
        messages.append(message)
    try:
        outputs = pipe(
            messages,
            max_new_tokens=12800,
        )
        response_message = [output[0]["generated_text"][-1]["content"] for output in outputs]
        return {"response": response_message}

    except Exception as e:
        raise Exception(str(e))


In [6]:
import pandas as pd
from collections import defaultdict

data_path = "C:/Users/cyon1/Desktop/LLM_cls_sum/data/article_samples.xlsx"
data = pd.read_excel(data_path)

original_documents = data["Articles"].tolist()

In [7]:
print(original_documents[:2])

['Every year in November, people look for bargains on Black Friday. But did you know that the same day is also Buy Nothing Day?\n\nDo the preparation task first. Then read the article and do the exercises.\n\nWhat is Black Friday?\nBlack Friday is the day after the American holiday of Thanksgiving, which is celebrated on the fourth Thursday of November. Because it is a holiday in the United States, it has long been a popular day for consumers to start shopping for Christmas. Over the last 20 years big retailers have started to offer discounts and bargains on this day, and it has become more and more popular. Last year, people in the USA spent an estimated $54.7 billion between Black Friday and Cyber Monday (the Monday after Thanksgiving, when people often buy more online). The idea of Black Friday has also spread around the world. For example, in 2017, people in the UK spent the equivalent of $10.3 billion, in Germany $7.6 billion and in France $6.2 billion.\n\nIs Black Friday out of c

In [8]:
summarized_documents = summarize_document(original_documents)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [9]:
classified_documents = classify_topic(original_documents)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [10]:
classified_documents

{'response': ["{\n'Politics': 0.00,\n'Economics': 0.00,\n'Social Issue': 0.40,\n'Culture & Arts': 0.00,\n'Natural Science': 0.00,\n'International Affairs': 0.00,\n'Technology & Internet': 0.00,\n'Health & Wellness': 0.00,\n'Travel': 0.00\n}",
  "Preparation complete.\n\nMain input text classification:\n\n{\n'Politics': 0.60,\n'International Affairs': 0.20,\n'Natural Science': 0.10,\n'Other': 0.10\n}",
  "{\n    'Culture & Arts': 0.20,\n    'History': 0.30,\n    'Social Issue': 0.20,\n    'Politics': 0.30\n}",
  "{\n    'Sports': 0.80,\n    'Culture & Arts': 0.15,\n    'Other': 0.05\n}",
  "{\n'Politics': 0.00,\n'Natural Science': 0.00,\n'Technology & Internet': 0.60,\n'Economics': 0.00,\n'Social Issue': 0.20,\n'Culture & Arts': 0.00,\n'International Affairs': 0.00,\n'Health & Wellness': 0.20,\n'Travel': 0.00\n}",
  "{\n    'Politics': 0.80,\n    'Culture & Arts': 0.05,\n    'Economics': 0.05,\n    'Social Issue': 0.05,\n    'Natural Science': 0.05,\n    'International Affairs': 0.05,\n

In [11]:
save_file = {
    "original_documents": original_documents,
    "summarized_documents": summarized_documents["response"],
    "classified_documents": classified_documents["response"]
}





In [12]:
df = pd.DataFrame(save_file)
df.to_excel("C:/Users/cyon1/Desktop/LLM_cls_sum/data/LLAMA3B_result.xlsx", index=False)
